In [1]:
#import OpenCV module
import cv2
#import os module for reading training data directories and paths
import os
#import numpy to convert python lists to numpy arrays as 
#it is needed by OpenCV face recognizers
import numpy as np

In [2]:
def name_map(x):
    if x=='sachin':
        return 0
    if x=='obama':
        return 1
    if x == 'messi':
        return 2

In [3]:
def get_name(x):
    if x==0:
        return 'sachin'
    if x==1:
        return 'obama'
    if x == 2:
        return 'messi'

In [4]:
#there is no label 0 in our training data so subject name for index/label 0 is empty
#subjects = ["sachin", "obama","messi"]

#function to detect face using OpenCV
def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV face detector, I am using haarcascade_frontalface_default
    
    face_cascade = cv2.CascadeClassifier('/home/luminar/Desktop/data/face/haarcascade_frontalface_default.xml')
    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [5]:
def prepare_training_data(data_folder_path):
    
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
            
        #------STEP-2--------
        #extract label number of subject from dir_name
        label = dir_name
        
        #build path of directory containin images for current subject subject
        #sample subject_dir_path = "data/DB/sachin"
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
        #------STEP-3--------
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            
            #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue;
            
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name

            #read image
            image = cv2.imread(image_path)
            
            #display an image window to show the image 
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            
            #detect face
            face, rect = detect_face(image)
            
            
            #------STEP-4--------
            #for the purpose of this tutorial
            #we will ignore faces that are not detected
            if face is not None:
                #add face to list of faces
                faces.append(face)
                #add label for this face
                labels.append(name_map(label))
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [6]:
print("Preparing data...")
faces, labels = prepare_training_data("/home/luminar/Desktop/data/DB")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing data...
Data prepared
Total faces:  17
Total labels:  17


In [7]:
faces

[array([[ 6,  6,  8, ..., 22, 15, 20],
        [ 6,  6,  8, ..., 18, 12, 18],
        [ 6,  6,  8, ..., 13,  9, 15],
        ...,
        [13, 11,  9, ..., 40, 47, 39],
        [10, 14, 16, ..., 45, 48, 31],
        [10, 12, 12, ..., 50, 40, 29]], dtype=uint8),
 array([[ 31,  23,  22, ..., 116, 119, 122],
        [ 29,  21,  23, ..., 119, 120, 122],
        [ 25,  18,  25, ..., 122, 123, 123],
        ...,
        [116, 117, 115, ..., 126, 126, 126],
        [118, 115, 108, ..., 126, 126, 126],
        [128, 109, 115, ..., 126, 126, 126]], dtype=uint8),
 array([[73, 51, 10, ..., 20, 15, 32],
        [72, 40, 15, ..., 19, 20, 33],
        [71, 36, 23, ...,  8, 18,  8],
        ...,
        [36, 36, 36, ...,  2,  7,  5],
        [36, 36, 36, ...,  7,  9, 21],
        [36, 36, 36, ..., 10, 12,  7]], dtype=uint8),
 array([[242, 242, 242, ...,  43,  32,  38],
        [242, 242, 242, ...,  42,  46,  44],
        [242, 241, 241, ...,  39,  47,  45],
        ...,
        [192, 186, 199, ...,  

In [8]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]

### Face Recognition

In [9]:
cv2.__version__

'4.3.0'

In [10]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

face_recognizer.train(faces, np.array(labels))

In [11]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), 2)

def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)

In [12]:
def predict(test_img):
    #make a copy of the image as we don't want to chang original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    try:
        label= face_recognizer.predict(face)
        #get name of respective label returned by face recognizer
        #label_text = subjects[label[0]]
        label_text = get_name(label[0])
        #draw a rectangle around face detected
        draw_rectangle(img, rect)
        #draw name of predicted person
        draw_text(img, label_text, rect[0], rect[1]-5)
    except:
        pass
    
    return img

In [ ]:
print("Predicting images...")

#load test images
test_img1 = cv2.imread("/home/luminar/Desktop/data/sachin.jpeg")
test_img2 = cv2.imread("/home/luminar/Desktop/data/obama.jpeg")
test_img3 = cv2.imread("/home/luminar/Desktop/data/messi.jpeg")

#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
predicted_img3 = predict(test_img3)
print("Prediction complete")

#display both images
cv2.imshow("Image1", predicted_img1)
cv2.imshow("Image2", predicted_img2)
cv2.imshow("Image3", predicted_img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

Predicting images...
Prediction complete


### Webcam

In [ ]:
video_capture = cv2.VideoCapture(0) #To Read video 0 for webcam else you can give the video path

In [ ]:
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    predicted_img1 = predict(frame)

    # Display the resulting frame
    cv2.imshow('Video', predicted_img1)

    if cv2.waitKey(1) & 0xFF == 27:
        break
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()